In [ ]:
! pip3 install pandas joblib scipy scikit-learn biopython imblearn propy3


In [1]:
import pandas as pd
import numpy as np
import joblib
import argparse
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.model_selection import GridSearchCV
from sklearn import metrics  # to calculate the accuracy of the model
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay, auc)
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn import tree
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from imblearn.under_sampling import RandomUnderSampler
import random

In [6]:
# INPUTs
common_dir = "/home/giulia/Workspace/PhytoPhD/effectors_analysis/classification/prediction_tools"
eff_predictions = f"{common_dir}/pred_eff_20220830/eff_clumps_feature_predictions20220830.tsv"
non_eff_predictions = f"{common_dir}/pred_non_eff_20220830/noneff_clumps_feature_preditions20220830.tsv"
print(pd.read_csv(eff_predictions, sep="\t").columns)
# OUTPUTs
model_output_path = "/home/giulia/Workspace/PhytoPhD/effectors_analysis/classification/random_forest/rf_clumps_20220830"

Index(['name', 'sequence length', 'signal peptide', 'aa in tr domain',
       'first 60 aa', 'prob N-in', 'transmembrane_domain', 'MobiDB-lite',
       'CLUMP0', 'CLUMP1', 'CLUMP2', 'CLUMP3', 'CLUMP4', 'CLUMP5', 'CLUMP6',
       'CLUMP7', 'CLUMP8', 'CLUMP9', 'CLUMP10'],
      dtype='object')


In [8]:
# DATASET 

original_protein_names = pd.concat([pd.read_csv(eff_predictions, sep="\t", header=0)[["name"]], 
                                    pd.read_csv(non_eff_predictions, sep="\t", header=0)[["name"]]], 
                                   ignore_index=True, join="inner")

## EFF PREDICTIONS DF for Random Forest
eff_pred_df = pd.read_csv(eff_predictions, sep="\t", header=0)
eff_pred_df["name"] = ["effectors"] * len(eff_pred_df)
# eff_pred_df = eff_pred_df.drop(columns=["ID", "organism"], axis=1)

## NON-EFF PREDICTIONS DF for Random Forest
non_eff_pred_df = pd.read_csv(non_eff_predictions, sep="\t", header=0)
non_eff_pred_df["name"] = ["non_effectors"] * len(non_eff_pred_df)

## COMBINE DFs (taking only effector motifs)
for col in eff_pred_df.columns[1:]:
    if col not in non_eff_pred_df.columns[1:]:
        # if an effector motifs is not present in non effectors, a new column will be added to the non_effector_df
        # having value False or 0 in this case
        non_eff_pred_df[col] = [0] * len(non_eff_pred_df)
    else:
        pass


# CONCAT AND CLEAN UP THE DATAFRAME FOR THE TRAINING
# "inner" will take only common cols
dataset = pd.concat([eff_pred_df, non_eff_pred_df], ignore_index=True, join="inner")
dataset.replace([True, False], ["1", "0"], inplace=True)

# drop the column/s having the same value for all samples == ZERO VARIANCE / ALMOST ZERO VARIANCE
# datset = dataset.drop(columns="EF_HAND_1", axis=1)

dataset 


,name,sequence length,signal peptide,aa in tr domain,first 60 aa,prob N-in,transmembrane_domain,MobiDB-lite,CLUMP0,CLUMP1,CLUMP2,CLUMP3,CLUMP4,CLUMP5,CLUMP6,CLUMP7,CLUMP8,CLUMP9,CLUMP10
0,effectors,125,0.696,20.43649,20.43648,0.72986,0,0,1,1,1,1,0,1,0,0,1,1,0
1,effectors,125,0.696,20.43649,20.43648,0.72986,0,0,1,1,1,1,0,1,0,0,1,1,0
2,effectors,125,0.768,20.54200,20.54197,0.88976,0,0,1,1,1,1,0,1,0,0,1,1,0
3,effectors,125,0.765,20.54231,20.54227,0.88976,0,0,2,1,1,1,0,1,0,0,1,1,0
4,effectors,125,0.765,20.54231,20.54227,0.88976,0,0,2,1,1,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,non_effectors,351,0.124,0.00201,0.00000,0.02435,1,0,3,0,0,1,0,1,0,0,0,0,0
332,non_effectors,507,0.104,0.01161,0.00025,0.00926,1,0,5,0,0,1,0,1,0,0,0,0,0
333,non_effectors,377,0.219,0.00057,0.00000,0.00874,1,0,2,0,0,2,0,0,0,0,0,0,0
334,non_effectors,264,0.224,0.00035,0.00000,0.02375,1,24,2,0,0,0,0,3,0,0,0,0,0


In [9]:
## CREATE THE RANDOM FOREST CLASSIFIER
### STRATIFIED SHUFFLE SPLIT K-fold CROSS-VALIDATION
#### ITERATIVELY ESTIMATE THE BEST NUMBER OF DECISION TREES IN THE FOREST FOR EACH FOLD

# shuffling is useful because positive and negative elements are ordered in the dataframe and this will help to
# randomly pick the elements for each fold

common_df = dataset

# skf5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# i = 1

# take trace of all evaluation parameters for each k-fold
kf_acc = []
kf_auc = []
kf_prec = []
kf_rec = []
kf_best_params = []
kf_pred_proba_test = []
kf_feat_imp = []

skf5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
i = 1


# SPLITTING
for train_index, test_index in skf5.split(common_df, list(common_df["name"])):
    
    x_train = common_df.iloc[train_index].loc[:, list(common_df.columns)[1:]]  # all cols except the name_col
    x_test = common_df.iloc[test_index][list(common_df.columns)[1:]]
    y_train = common_df["name"].iloc[train_index]  # only the name_col
    y_test = common_df["name"].iloc[test_index]

    # kth-fold RANDOM FOREST
    clf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators = number of trees in the forest
    
    ## SELECT BEST n_estimators on the TRAINING SET
    print("BEGIN best parameters-n_trees selection for Random Forest")
    params_to_test = {"n_estimators": [50, 75, 100, 1000, 5000]}
    grid_search = GridSearchCV(clf, params_to_test, n_jobs=4)
    grid_search.fit(x_train, y_train)
    print("FINISH - best parameters-n_trees selection")
    best_params = grid_search.best_params_
    clf = RandomForestClassifier(**best_params, random_state=42, max_features="sqrt")
    kf_best_params.append(best_params["n_estimators"])
    print(clf.get_params())
    ## TRAIN THE MODEL USING kth-FOLD TRAINING SETS
    clf.fit(x_train, y_train)
    
       
    ## APPLY THE MODEL ON TEST SET
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)
    kf_pred_proba_test.append([y_prob])

    ## ACCURACY
    kth_accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"Training phase accuracy for the fold no. {i}: {kth_accuracy}")
    kf_acc.append(kth_accuracy)
    print(f"acc_train: {clf.score(x_train, y_train)}\nacc_test: {clf.score(x_test, y_test)}")

    ## F-measure
    f_score = metrics.f1_score(y_test, y_pred, average="macro")
    print(f"\nF-Measure:\t{f_score}")
    
    ### predict probability
    y_pred_prob = clf.predict_proba(x_test)

    ## PRECISION AND RECALL
    y_test_hot_encode = [0 if el == "non_effectors" else 1 for el in y_test]
    y_pred_hot_encode = [0 if el == "non_effectors" else 1 for el in y_pred]
    precision_perc = metrics.precision_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    recall_perc = metrics.recall_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    precision, recall, _ = precision_recall_curve(y_test_hot_encode, y_pred_hot_encode, pos_label=1)

    print(f"Precision (TP/TP+FP) on training phase: {precision_perc}")
    print(f"Recall (TP/TP+FN) on training phase: {recall_perc}")
    print(f"Precision coordinates: {precision}\nRecall(Sensitivity) coordinates: {recall}")

    kf_prec.append(precision_perc)
    kf_rec.append(recall_perc)
    
    ## Area Under the Curve
    pr_auc = auc(recall, precision)
    kf_auc.append(pr_auc)
    # disp = PrecisionRecallDisplay(precision=precision, recall=recall)
    # disp.plot()
    print(f"Area under the curve is: {pr_auc}")

    ## CONFUSION MATRIX
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print(f"\nconfusion matrix on k-fold test set\n{conf_matrix}")

    ## FEATURE IMPORTANCE
    ### MDI
    feat_imp = pd.Series(clf.feature_importances_, index=list(common_df.columns)[1:]).sort_values(ascending=False)
    print(f"Feature importance:\n{feat_imp}")
    kf_feat_imp.append(feat_imp.to_dict())

    
    
#     ### PERMUTATION IMPORTANCE
#     perm_imp = permutation_importance(clf, x_train, y_train, 
#                                        n_repeats=10, random_state=42, n_jobs=2, scoring='accuracy')
#     mean_perm_imp = pd.Series(perm_imp.importances_mean, index=x_train.columns).sort_values(ascending=False)
#     print(f" Permutation-based Feature Importance:\n{mean_perm_imp}")
#     for i in perm_imp.importances_mean.argsort()[::-1]:
#         print(f"{perm_imp.importances_mean[i]}\t+/- {perm_imp.importances_std[i]}")
    
    i += 1
    

print(f"Averaged accuracy for {i-1}-fold cross validation: {np.mean(kf_acc)*100}%")
print(f"Averaged precision for {i-1}-fold cross validation: {np.mean(kf_prec)*100}%")
print(f"Averaged recall for {i-1}-fold cross validation: {np.mean(kf_rec)*100}%")
print(f"Averaged best number of tree in the forest: {np.mean(kf_best_params)}")



BEGIN best parameters-n_trees selection for Random Forest
FINISH - best parameters-n_trees selection
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Training phase accuracy for the fold no. 1: 1.0
acc_train: 1.0
acc_test: 1.0

F-Measure:	1.0
Precision (TP/TP+FP) on training phase: 1.0
Recall (TP/TP+FN) on training phase: 1.0
Precision coordinates: [0.26470588 1.         1.        ]
Recall(Sensitivity) coordinates: [1. 1. 0.]
Area under the curve is: 1.0

confusion matrix on k-fold test set
[[18  0]
 [ 0 50]]
Feature importance:
first 60 aa             0.265207
aa in tr domain         0.249424
signal peptide          0.166246
transmembrane_domain    0

In [10]:
av_feat_imp = {}
for d in kf_feat_imp:
    for k,v in d.items():
        if k not in list(av_feat_imp.keys()):
            av_feat_imp[k] = [v]
        else:
            av_feat_imp[k] += [v]
av_d = {}            
for k, v in list(av_feat_imp.items()):
    av_d[k] = np.mean(v)
    
av_d_df = pd.Series(av_d, index=list(av_d.keys()))
av_d_df.to_csv(f"{model_output_path}/feature_importance.tsv", sep="\t")

In [ ]:
# print(f"Prediction probability for each fold\n{pred_proba_df}")
print(kf_pred_proba_test)

In [11]:
# RANDOM FOREST WITH BEST PARAMETERS TO THE ENTIRE SET OF DATA
# rf_classifier = RandomForestClassifier(n_estimators=int(np.mean(kf_best_params)), random_state=42)

d_params = {}
for el in clf.get_params():
    if el == "n_estimators":
        d_params["n_estimators"] = int(np.mean(kf_best_params))
    else:
        d_params[el] = clf.get_params()[el]

rf_classifier = RandomForestClassifier(**d_params)
rf_classifier.fit(common_df[list(common_df.columns)[1:]], common_df["name"])

# SAVE THE MODEL
joblib.dump(rf_classifier, f"{model_output_path}/rf_best_model.pkl")

['/home/giulia/Workspace/PhytoPhD/effectors_analysis/classification/random_forest/rf_clumps_20220830/rf_best_model.pkl']

In [14]:
# PERFORMANCE ON RANDOM LABELING
acc_random = []
for i in range(100):
    random.shuffle(x := list(common_df["name"]))
    pred = rf_classifier.predict(common_df[list(common_df)[1:]])
    acc = metrics.accuracy_score(x, pred)
    acc_random.append(acc)
    i += 1

print(np.mean(acc_random))

0.6138095238095238


In [15]:
# CALCUALTE PROBAILITY OF EACH CLASS PREDICTION 
y_proba = rf_classifier.predict_proba(common_df[list(common_df.columns)[1:]])
pred_proba_df = pd.DataFrame(y_proba, index=common_df["name"], columns=rf_classifier.classes_)
print(pred_proba_df)
pred_proba_df.to_csv(f"{model_output_path}/rf_best_model_predictions_probability.tsv", sep="\t", index=False)


               effectors  non_effectors
name                                   
effectors            1.0            0.0
effectors            1.0            0.0
effectors            1.0            0.0
effectors            1.0            0.0
effectors            1.0            0.0
...                  ...            ...
non_effectors        0.0            1.0
non_effectors        0.0            1.0
non_effectors        0.0            1.0
non_effectors        0.0            1.0
non_effectors        0.0            1.0

[336 rows x 2 columns]


In [ ]:
# PERMUTATION-BASED FEATURE IMPORTANCE

per_importance = permutation_importance(clf, common_df[list(common_df.columns)[1:]], common_df["name"], 
                                       n_repeats=30, random_state=42, scoring='accuracy')
for i in per_importance.importances_mean.argsort()[::-1]:
    print(f"{per_importance.importances_mean[i]}\t+/- {per_importance.importances_std[i]}")
# imp_importance = pd.Series(clf.feature_importances_, index=list(common_df.columns)[1:]).sort_values(ascending=False)
imp_importance = rf_classifier.feature_importances_
print(imp_importance)

In [ ]:
# OVERFIT THE MODEL TO SEE IF THE PERMUTATION FEATURE IMPORTANCE CHANGE 
over_dataset = dataset[['name', 'sequence length', 'signal peptide', 'transmembrane domain',
       'Phob signal peptide', 'Phob transmembrane domain', 'h_bin1', 'h_bin2', 'h_bin3']]
over_dataset
